This repo is currently modified to inference on CPU environment. To implement on GPU, must edit some files from current repository. (Original DPR repo is based on GPU implementation.)

1. In file `DPR/conf/dense_retriever.yaml`, change `no_cuda` to `False`.
2. Edit `DPR/dense_retriever.py` to add `.cuda()` to lines 83 and 84 with variables `q_ids_batch` and `q_seg_batch`. 

In [ ]:
# cd DPR
!git clone https://github.com/facebookresearch/DPR.git

In [ ]:
# list of necessary downloads for inference
# %%bash
# python dpr/data/download_data.py --resource data.wikipedia_split.psgs_w100
# python dpr/data/download_data.py --resource data.retriever.nq
# python dpr/data/download_data.py --resource data.retriever.qas.nq
# python dpr/data/download_data.py --resource data.retriever_results.nq.single-adv-hn.wikipedia_passages
# python dpr/data/download_data.py --resource checkpoint.retriever.single.nq.bert-base-encoder

In [18]:
!python dense_retriever.py \
model_file='downloads/checkpoint/retriever/single/nq/bert-base-encoder.cp' \
qa_dataset=nq_test \
out_file='./result_dense_retriever/' \
ctx_datatsets=dpr_wiki \
encoded_ctx_files="downloads/data/retriever_results/nq/single-adv-hn/wikipedia_passages_*"

### Inference using EfficientQA

In [ ]:
# more necessary downloads
# %%bash
# python dpr/data/download_data.py --resource checkpoint.retriever.multiset.bert-base-encoder
# python dpr/data/download_data.py --resource checkpoint.reader.nq-single.hf-bert-base

In [ ]:
!git clone https://github.com/efficientqa/retrieval-based-baselines.git && cd retrieval-based-baselines && pip3 install -r requirements.txt

For reference

```
python3 run_inference.py \
  --qa_file NQ-open.dev.jsonl \ # data file with questions
  --retrieval_type {tfidf|dpr} \ # which retrieval to use
  --db_path ${base_dir}/data/wikipedia_split/{psgs_w100.tsv|psgs_w100_subset.tsv} \
  --tfidf_path ${tfidf_index} \ # only matters for TFIDF retrieval
  --dpr_model_file ${dpr_retrieval_checkpoint} \ # only matters for dpr retrieval
  --dense_index_path ${dpr_index} \ # only matters for dpr retrieval
  --model_file ${reader_checkpoint} \ # path to the reader checkpoint
  --dev_batch_size 8 \ # 8 is good for one 32gb GPU
  --pretrained_model_cfg bert-base-uncased --encoder_model_type hf_bert --do_lower_case \
  --sequence_length 350 --eval_top_docs 10 20 40 50 80 100 \
  --passages_per_question_predict 100 \ # 100 for TFIDF, 40 for DPR
  --prediction_results_file dev_predictions.json # path to save predictions; comparable to the official evaluation script
  ```

In [ ]:
!python run_inference.py \
  --qa_file DPR/downloads/data/retriever/qas/nq-test.csv \
  --retrieval_type dpr \
  --db_path DPR/downloads/data/wikipedia_split/psgs_w100.tsv \
  --dpr_model_file DPR/downloads/checkpoint/retriever/multiset/bert-base-encoder.cp \
  --dense_index_path DPR/downloads/indexes/single/nq/full/index.dpr \
  --model_file DPR/downloads/checkpoint/reader/nq-single/hf-bert-base.cp \
  --dev_batch_size 1 \
  --pretrained_model_cfg bert-base-uncased --encoder_model_type hf_bert --do_lower_case \
  --sequence_length 350 --eval_top_docs 10 \
  --passages_per_question_predict 40 \
  --prediction_results_file dev_predictions.json